In [1]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from multiprocessing import Pool, cpu_count
import functools

# Parse pathway features

In [2]:
entrez2symbol = {}
with open('data/Homo_sapiens.gene_info') as f:
    for line in f.read().splitlines():
        row = line.split('\t')
        entrez2symbol[row[1]]=row[2]

These are the pathways related to cancer hallmarks, which will be used as edge features.  
Referance: http://www.cell.com/abstract/S0092-8674(11)00127-9

In [3]:
gene2pathway = {}
pathways = set()
with open('data/hallmarks.txt') as f:
    for line in f.read().splitlines():
        row = line.split('\t')
        if len(row) > 1:
            pathway = row[0].split('|')[1]
            pathways.add(pathway)
            for entrez in row[2:]:
                if entrez not in entrez2symbol:
                    continue
                gene = entrez2symbol[entrez]
                if gene not in gene2pathway:
                    gene2pathway[gene]=set()
                gene2pathway[gene].add(pathway)

# Parse TCGA BRCA subtypes

In [4]:
pat2subtype = {}
def parse_subtypes(fn, pat2subtype):
    with open(fn) as f:
        for line in f.read().rstrip().splitlines()[1:]:
            row = line.split("\t")
            pat = row[0][:12]
            if len(row[0])>12:
                tissue_code = row[0][13:15]
                if int(tissue_code) >= 10:
                    continue
            # Remove "Normal" subtype
            if row[1] in ['NA', 'Normal']:
                continue
            pat2subtype[pat] = row[1]
        return pat2subtype
pat2subtype = parse_subtypes('data/TCGABRCA2PAM50_nature547.txt', pat2subtype)
pat2subtype = parse_subtypes('data/TCGABRCA2PAM50_nature522.txt', pat2subtype)
pat2subtype = parse_subtypes('data/TCGABRCA2PAM50_cell871.txt', pat2subtype)
pat2subtype = parse_subtypes('data/TCGABRCA2PAM50_cell817.txt', pat2subtype)

# Filter by cancer genes (optional)

In [5]:
oncogene_tsg={}
with open('data/oncogene_tsg.txt') as f:
    for line in f.read().rstrip().splitlines():
        row = line.split("\t")
        oncogene_tsg[row[0]] = row[1]
with open('data/oncogene_tsg_BRCA.txt') as f:
    for line in f.read().rstrip().splitlines():
        row = line.split("\t")
        oncogene_tsg[row[0]] = row[1]

In [6]:
cancergenes = set() | set(oncogene_tsg.keys()) | set(gene2pathway.keys())
with open('data/CGs.txt') as f:
    for line in f.read().rstrip().splitlines():
        cancergenes.add(line)
with open('data/cancer_gene_census.csv') as f:
    for line in f.read().rstrip().splitlines()[1:]:
        row = line.split(',')
        cancergenes.add(row[0])
len(cancergenes)

2317

# Parse mutation and CNA profile

Genes were classified as wild type (0) or altered (1) in each of the tumors with alterations defined as follows:
1. Most oncogenes (e.g., EGFR) were considered altered (activated) if impacted by a missense mutation, in-frame indel or copy number amplification. 
2. For the subset of oncogenes typically altered only by amplification (CCND1, LMO1, MDM2, MDM4, MYC, MYCL, MYCN, NCOA3, NKX2-1 and SKP2), only copy number amplifications were considered as alterations and not SNVs or indels. 
3. All other genes including tumor suppressors (e.g., CDKN2A) were considered altered (inactivated) if there was any type of non-silent mutation or a copy number deletion.

In [7]:
def parse_maf(fn, oncogene_tsg):
    df = pd.read_table(fn, low_memory=False)
    df = df.loc[(df.loc[:,'is_flank']==0) & (df.loc[:,'is_silent']==0),:]
    df['pat'] = df.loc[:,'Tumor_Sample_Barcode'].str[:12]
    
    filter_rows = []
    genes = set()
    for index, row in df.iterrows():
        gene = row['Hugo_Symbol']
        # (Optional) filter cancer genes
        if gene not in cancergenes:
            continue
        genes.add(gene)
        VC = row['Variant_Classification']
        if gene in oncogene_tsg:
            if oncogene_tsg[gene] in ['Oncogene']:
                if VC not in ['Missense_Mutation', 'In_Frame_Del', 'In_Frame_Ins', 'De_novo_Start_InFrame']:
                    filter_rows.append(index)
            if oncogene_tsg[gene] == 'Amplification_Oncogene':
                filter_rows.append(index)
    df = df.drop(filter_rows)
    
    df = df.loc[:,['pat','Hugo_Symbol']]
    df['counter'] = 1
    df.set_index(['pat','Hugo_Symbol'], inplace=True)
    df = df.counter.groupby(level=[0,1]).min().unstack()
    df.fillna(0, inplace=True)
    return df, genes

In [8]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2016_01_28/analyses/BRCA/Mutation_Assessor/BRCA-TP.maf.annotated'
df_mut, genes = parse_maf(fn, oncogene_tsg)

In [9]:
def parse_CNA(fn, genes):
    df = pd.read_table(fn,low_memory=False,index_col=0)
    df = df[df.index.isin(genes)]
    # (Optional) filter cancer genes
    df = df[df.index.isin(cancergenes)]
    df = df.iloc[:,2:]
    df = (df/2.).round(0)
    df.columns = df.columns.str[:12]
    
    nonOncogene_rows = []
    for index, row in df.iterrows():
        gene = index
        if not (gene in oncogene_tsg and oncogene_tsg[gene] in ['Oncogene', 'Amplification_Oncogene']):
            nonOncogene_rows.append(index)
    df.loc[nonOncogene_rows,:] = df.loc[nonOncogene_rows,:] * (-1)
    df = df.clip(lower=0)
    return df

In [10]:
coding_genes = set()
with open("/cellar/data/users/wzhang1984/bcbio/genomes/Hsapiens/GRCh37/rnaseq-2014-07-14/ref-transcripts.gtf") as f:
    for line in f.read().splitlines():
        row = line.split("\t")
        if row[1] != "protein_coding" or row[2] != "transcript":
            continue
        gene_name = row[-1].split('gene_name "')[1].split('"')[0]
        if gene_name:
            coding_genes.add(gene_name)

In [11]:
coding_genes = coding_genes | genes

In [12]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2016_01_28/analyses/BRCA/CopyNumber_Gistic2/all_thresholded.by_genes.txt'
df_CNA = parse_CNA(fn, coding_genes)

In [13]:
df_mut_CNA = pd.concat([df_mut, df_CNA.transpose()], axis=1,
                       join='inner').transpose().groupby(level=0).sum().clip(upper=1.).transpose()
df_mut_CNA = df_mut_CNA.loc[set(df_mut.index) & set(df_CNA.columns) & set(pat2subtype.keys()),df_CNA.index]

In [14]:
df_mut_CNA = df_mut_CNA[(df_mut_CNA.T != 0).any()] # remove samples with all zeros
df_mut_CNA.shape

(867, 2247)

# Seperating training , validation and testing samples

In [105]:
training_set = df_mut_CNA.sample(frac=2./3)
training_set.sort_index(inplace=True)

In [106]:
validation_set = df_mut_CNA.drop(training_set.index).sample(frac=1./2)
validation_set.sort_index(inplace=True)

In [107]:
testing_set = df_mut_CNA.drop(training_set.index).drop(validation_set.index)
testing_set.sort_index(inplace=True)

# Or load existing datasets (Optional)

In [15]:
training_set = pd.read_table('data/archive/08182017/BRCA_training_data.txt', index_col=0)
validation_set = pd.read_table('data/archive/08182017/BRCA_validation_data.txt', index_col=0)
testing_set = pd.read_table('data/archive/08182017/BRCA_testing_data.txt', index_col=0)
training_set = df_mut_CNA.loc[training_set.index,:]
validation_set = df_mut_CNA.loc[validation_set.index,:]
testing_set = df_mut_CNA.loc[testing_set.index,:]

# Fisher test against subtypes (Optional)

In [84]:
training_pat2subtype = training_set.index.to_series().map(pat2subtype)
gene2fisherp = {}
for gene in training_set.columns:
    gene2fisherp[gene] = 0.
    for subtype in training_pat2subtype.unique():
        tab = pd.crosstab(training_set[gene]>0, training_pat2subtype==subtype)
        if tab.shape != (2, 2):
            continue
        oddsratio, pvalue = fisher_exact(tab, alternative='greater')
        logp = np.log10(pvalue)
        gene2fisherp[gene] = max(-logp, gene2fisherp[gene])
        if pvalue < 0.05/len(training_set.index):
            print gene, subtype, oddsratio, pvalue

PIK3CA LumA 2.27310855263 2.32810510033e-06
FST Basal 18.8764044944 7.76305289558e-05
PIK3R1 Basal 9.22480620155 3.50117992123e-05
CDK7 Basal 18.8764044944 7.76305289558e-05
GTF2H2 Basal 18.8764044944 7.76305289558e-05
SV2C Basal 14.5151515152 4.77568792893e-05
F2R Basal 18.8764044944 7.76305289558e-05
MYC Basal 4.0060851927 7.64961832658e-09
JAK2 Basal 11.3333333333 1.8486829055e-06
ABL1 Her2 32.625 1.2344499266e-05
CCND1 LumB 2.51196172249 6.13728097649e-05
AQR Basal 18.8764044944 7.76305289558e-05
BUB1B Basal inf 2.89081864236e-06
CASC5 Basal 18.8764044944 7.76305289558e-05
CDH1 LumA 5.02258592471 1.96424966692e-08
TP53 Basal 10.3643709119 2.28918000204e-22
TP53 Her2 5.47769443348 9.10618167532e-09
ERBB2 Her2 20.2522727273 1.36195464324e-21
SPOP Her2 5.54196301565 4.2783176358e-06
STK11 Basal 21.8181818182 1.50940436589e-05
CCNE1 Basal 8.05853658537 2.38604254457e-06


# Parse PathwayCommons

In [17]:
PathwayCommons = pd.read_table('/cellar/users/wzhang1984/Data/PathwayCommons/PathwayCommons9.All.hgnc.txt')

In [18]:
PathwayCommons = PathwayCommons.loc[PathwayCommons.loc[:,'INTERACTION_TYPE'].isin(['controls-state-change-of',
                                                                                   'controls-transport-of',
                                                                                   'controls-phosphorylation-of',
                                                                                   'controls-expression-of',
                                                                                   'catalysis-precedes',
                                                                                   'in-complex-with',
                                                                                   'interacts-with', 
                                                                                   'neighbor-of']),:]

In [62]:
def parse_edge_features(mutrates, df):
    edge2features = {}
    for index, row in df.iterrows():
        g1 = row['PARTICIPANT_A']
        g2 = row['PARTICIPANT_B']
        
        # (Optionally) filter by cancer genes or pathways
        if not (g1 in cancergenes and g2 in cancergenes):
            continue
            
        ty = row['INTERACTION_TYPE']
        ty_d = ty + '_d'
        ty_rev = ty + '_rev'
        sources = row['INTERACTION_DATA_SOURCE'].split(';')
        edge = g1 + '\t' + g2
        edge_rev = g2 + '\t' + g1
        if edge not in edge2features:
            edge2features[edge] = {'gene 1':g1, 'gene 2':g2}
        if edge_rev not in edge2features:
            edge2features[edge_rev] = {'gene 1':g2, 'gene 2':g1}

        # Parse edge type features
        if ty not in ['in-complex-with','interacts-with','neighbor-of']:
            edge2features[edge][ty_d] = 1.
            edge2features[edge_rev][ty_rev] = 1.
        else:
            edge2features[edge][ty] = 1.
            edge2features[edge_rev][ty] = 1.

        # Parse edge source features
        for source in sources:
            edge2features[edge][source] = 1.
            edge2features[edge_rev][source] = 1.

        # Parse pathway features. 
        # If one node is in the pathway, the score is 0.5
        # If both nodes are in the pathway, the score is 1
        if g1 in gene2pathway or g2 in gene2pathway:
            for pathway in pathways:
                edge2features[edge][pathway] = 0.
                edge2features[edge_rev][pathway] = 0.
            for g in [g1, g2]:
                if g in gene2pathway:
                    for pathway in gene2pathway[g]:
                        edge2features[edge][pathway] += 0.5
                        edge2features[edge_rev][pathway] += 0.5

        # Parse mutation features from the training set
        # Calculate mutation rates
        mutrate_g1 = 0
        mutrate_g2 = 0
        if g1 in training_set:
            mutrate_g1 = mutrates.loc[g1]
        if g2 in training_set:
            mutrate_g2 = mutrates.loc[g2]
        edge2features[edge]['mutrate_source'] = mutrate_g1
        edge2features[edge]['mutrate_target'] = mutrate_g2
        edge2features[edge_rev]['mutrate_source'] = mutrate_g2
        edge2features[edge_rev]['mutrate_target'] = mutrate_g1

        # (optional) add fisher test against subtypes
        if g1 in gene2fisherp:
            edge2features[edge]['fisherp_source'] = gene2fisherp[g1]
            edge2features[edge_rev]['fisherp_target'] = gene2fisherp[g1]
        if g2 in gene2fisherp:
            edge2features[edge]['fisherp_target'] = gene2fisherp[g2]
            edge2features[edge_rev]['fisherp_source'] = gene2fisherp[g2]

        # Calculate mutual exclusivity / co-occurrence
        ME = 0.
        if g1 in training_set and g2 in training_set:
            if training_set.loc[:,g1].sum() < 7 or training_set.loc[:,g2].sum() < 7:
                continue
            tab = pd.crosstab(training_set.loc[:,g1],training_set.loc[:,g2])
            if tab.shape != (2, 2):
                continue
            if tab.iloc[1,1] * tab.iloc[0,0] >= tab.iloc[0,1] * tab.iloc[1,0]:
                continue
            oddsratio, pvalue = fisher_exact(tab, alternative='less')
            logp = np.log10(pvalue)
            ME = -logp
            if pvalue < 0.05:
                print g1, g2, oddsratio, pvalue
        edge2features[edge]['mutual_exclusive'] = ME
        edge2features[edge_rev]['mutual_exclusive'] = ME
            
    return edge2features

In [63]:
training_set_mutrate = training_set.sum() / training_set.shape[0]

n_processes = cpu_count()
pool = Pool(processes=n_processes)

df_split = np.array_split(PathwayCommons, n_processes, axis=0)
parse_edge_features_partial = functools.partial(parse_edge_features, training_set_mutrate)
edge2features_list = pool.map(parse_edge_features_partial, df_split)

pool.close()
pool.join()

MDM4 TP53 0.479289940828 0.00541029222261
MDM4 TP53 0.479289940828 0.00541029222261
MDM4 TP53 0.479289940828 0.00541029222261
MDM4 TP53 0.479289940828 0.00541029222261
MYC RUNX1 0.270512820513 0.0405758287654
CCND1 RB1 0.315750915751 0.0284796975281
CCND1 RB1 0.315750915751 0.0284796975281
CCND1 RB1 0.315750915751 0.0284796975281
PTEN PIK3CA 0.266976744186 0.000334407812184
PTEN PIK3CA 0.266976744186 0.000334407812184
CDH1 IGF1R 0.18759936407 0.0440050002472
TP53 MDM4 0.479289940828 0.00541029222261
ERBB2 PTEN 0.200091575092 0.00670309323965
PIK3CA PTEN 0.266976744186 0.000334407812184
PIK3CA RB1 0.465564738292 0.0288895694816
PIK3R1 CCND1 0.0 0.0347999872954


In [64]:
edge2features = {}
for i in range(len(edge2features_list)):
    edge2features.update(edge2features_list[i])

In [65]:
edge2features_df = pd.DataFrame.from_dict(edge2features, orient='index')

In [66]:
edge2features_df = edge2features_df.fillna(0).sort_index(1)
cols = list(edge2features_df)
cols.insert(0, cols.pop(cols.index('gene 2')))
cols.insert(0, cols.pop(cols.index('gene 1')))
edge2features_df = edge2features_df[cols]

# Add high mutrate and high degree features (Optional)

In [67]:
# Top 5 genes with the highest degree
# geneset = set(edge2features_df.groupby('gene 1')['gene 2'].count().sort_values()[-5:].index)
# Top 5 genes with the highset mutation rate
geneset = set(training_set.sum().sort_values()[-5:].index)

In [68]:
for gene in sorted(geneset):
    edge2features_df[gene+'_source'] = (edge2features_df['gene 1']==gene).astype(int)
    edge2features_df[gene+'_target'] = (edge2features_df['gene 2']==gene).astype(int)

In [69]:
edge2features_df = edge2features_df.fillna(0).sort_index(1)
cols = list(edge2features_df)
cols.insert(0, cols.pop(cols.index('gene 2')))
cols.insert(0, cols.pop(cols.index('gene 1')))
edge2features_df = edge2features_df[cols]

# Feature scaling (Optional)

In [70]:
edge2features_df.iloc[:,2:] = edge2features_df.iloc[:,2:].divide(edge2features_df.iloc[:,2:].max())

# Output dataframes to files

In [132]:
training_set.to_csv('data/BRCA_training_data.txt', sep='\t')
validation_set.to_csv('data/BRCA_validation_data.txt', sep='\t')
testing_set.to_csv('data/BRCA_testing_data.txt', sep='\t')

In [133]:
training_set.index.to_series().map(pat2subtype).to_csv('data/BRCA_training_lables.txt', sep='\t')
validation_set.index.to_series().map(pat2subtype).to_csv('data/BRCA_validation_lables.txt', sep='\t')
testing_set.index.to_series().map(pat2subtype).to_csv('data/BRCA_testing_lables.txt', sep='\t')

In [71]:
edge2features_df.to_csv('data/BRCA_edge2features.txt', sep='\t', header=False, index=False)

In [72]:
with open('data/BRCA_feature_names.txt', 'w') as f:
    f.write('\n'.join(edge2features_df.columns[2:]))

## Select features (optional)

In [40]:
features_sig = set()
with open('data/BRCA_edge_feature_weights_9.txt') as f:
    for line in f.read().rstrip().splitlines()[1:]:
        w = line.split('\t')
        if float(w[1]) > 0.9:
            features_sig.add(w[0])

In [41]:
features_sorted = sorted(features_union)
edge2features_line_out = ''
for gene_pair in sorted(edge2features_union):
    edge2features_line_out += gene_pair
    for feature in features_sorted:
        if feature not in features_sig:
            continue
        if feature in edge2features_union[gene_pair]:
            edge2features_line_out += '\t{}'.format(edge2features_union[gene_pair][feature])
        else:
            edge2features_line_out += '\t0.0'
    edge2features_line_out += '\n'

In [42]:
with open('data/BRCA_edge2features_logistic.txt', 'w') as f:
    f.write(edge2features_line_out)
with open('data/BRCA_feature_names_logistic.txt', 'w') as f:
     f.write('\n'.join(sorted(set(features_sorted) & features_sig)))

# Scratch

In [73]:
edge2features_df.loc['PIK3CA\tTP53',:]

gene 1                                      PIK3CA
gene 2                                        TP53
Adherens junction                                0
Apoptosis                                        1
B cell receptor signaling pathway              0.5
BIND                                             0
Base excision repair                             0
BioGRID                                          0
CCND1_source                                     0
CCND1_target                                     0
CORUM                                            0
CTD                                              1
Cell cycle                                     0.5
Cytokine-cytokine receptor interaction           0
DIP                                              0
ECM-receptor interaction                         0
ERBB2_source                                     0
ERBB2_target                                     0
ESC proliferation                              0.5
Epithelial-mesenchymal transiti

In [207]:
training_set.sum().sort_values()[::-1]

Gene Symbol
PIK3CA       207.0
TP53         186.0
MYC          132.0
CCND1        108.0
ERBB2         93.0
AKT3          86.0
MDM4          85.0
H3F3A         83.0
CDH1          69.0
IKBKB         64.0
GATA3         63.0
SPOP          53.0
PTEN          50.0
MAP3K1        50.0
PAK1          48.0
RB1           42.0
GNAS          40.0
IGF1R         38.0
MSR1          38.0
MAP2K4        38.0
TNFRSF10A     38.0
PPP3CC        37.0
PCM1          37.0
TNFRSF10C     37.0
TNFRSF10D     37.0
TNFRSF10B     37.0
SRSF2         37.0
FZD3          36.0
PTK2B         35.0
NEIL2         35.0
             ...  
AGER           0.0
AIF1           0.0
C4B            0.0
CSNK2B         0.0
HSPA1A         0.0
PRKACA         0.0
SLC27A1        0.0
NTF4           0.0
CXCL9          0.0
LIAS           0.0
PF4V1          0.0
CXCL1          0.0
PPBP           0.0
CXCL5          0.0
CXCL3          0.0
CXCL2          0.0
EREG           0.0
BTC            0.0
CXCL11         0.0
JUND           0.0
CXCL13         0.0
